## Calculate occupancy of units based on admissions and length of stay

In [1]:
import numpy as np
import pandas as pd

In [2]:
admissions = pd.read_csv('admissions.csv')
hospital_info =  pd.read_csv('hospitals.csv')
preferred_unit = pd.read_csv('pref_unit.csv')

In [3]:
admissions.head()

,LSOA,CCG,stp,nhs_region,admissions
0,Bromley 007B,NHS Bromley CCG,South East London,London,3.43
1,Bromley 016C,NHS Bromley CCG,South East London,London,1.37
2,Bromley 020A,NHS Bromley CCG,South East London,London,6.86
3,Bromley 022A,NHS Bromley CCG,South East London,London,3.43
4,Bromley 016D,NHS Bromley CCG,South East London,London,4.11


In [4]:
hospital_info.head()

,Unit,Postcode,Use,los_mean,Capacity
0,Barnet General SU,EN53DJ,1,27.3,9999
1,Charing Cross SU,W68RF,1,40.0,9999
2,Chelsea & Wminster SU,SW109NH,1,29.5,9999
3,Croydon SU,CR77YE,1,39.6,9999
4,Hillingdon SU,UB83NN,1,33.7,9999


In [5]:
preferred_unit.head()

,LSOA,LSOA01,WD03CD,WD03NM,data_hosp,Preferred_unit_name,Preferred_unit_postcode
0,Barking and Dagenham 001A,Barking and Dagenham 001A,00ABGA,Chadwell Heath,Queen's Hospital,Queens Romford SU,RM70AG
1,Barking and Dagenham 001B,Barking and Dagenham 001B,00ABGA,Chadwell Heath,Queen's Hospital,Queens Romford SU,RM70AG
2,Barking and Dagenham 001C,Barking and Dagenham 001C,00ABGA,Chadwell Heath,Queen's Hospital,Queens Romford SU,RM70AG
3,Barking and Dagenham 001D,Barking and Dagenham 001D,00ABGA,Chadwell Heath,Queen's Hospital,Queens Romford SU,RM70AG
4,Barking and Dagenham 002A,Barking and Dagenham 002A,00ABGA,Chadwell Heath,Queen's Hospital,Queens Romford SU,RM70AG


## Add preferred unit to admissions

In [6]:
admissions_with_preferred = admissions.merge(preferred_unit, left_on='LSOA', right_on='LSOA', how='left')

In [7]:
admissions_with_preferred.head()

,LSOA,CCG,stp,nhs_region,admissions,LSOA01,WD03CD,WD03NM,data_hosp,Preferred_unit_name,Preferred_unit_postcode
0,Bromley 007B,NHS Bromley CCG,South East London,London,3.43,Bromley 007B,00AFGD,Bickley,Princess Royal University Hospital,Princess Royal SU,BR68ND
1,Bromley 016C,NHS Bromley CCG,South East London,London,1.37,Bromley 016C,00AFGD,Bickley,Princess Royal University Hospital,Princess Royal SU,BR68ND
2,Bromley 020A,NHS Bromley CCG,South East London,London,6.86,Bromley 020A,00AFGD,Bickley,Princess Royal University Hospital,Princess Royal SU,BR68ND
3,Bromley 022A,NHS Bromley CCG,South East London,London,3.43,Bromley 022A,00AFGD,Bickley,Princess Royal University Hospital,Princess Royal SU,BR68ND
4,Bromley 016D,NHS Bromley CCG,South East London,London,4.11,Bromley 016D,00AFGD,Bickley,Princess Royal University Hospital,Princess Royal SU,BR68ND


In [8]:
admissions_with_preferred.to_csv('admissions_with_preferred_unit.csv')

## Sum admissions by hospital

In [9]:
summary = pd.DataFrame()
summary['admissions'] = \
    admissions_with_preferred.groupby('Preferred_unit_name').sum()['admissions']

In [10]:
summary.head()

,admissions
Preferred_unit_name,
Barnet General SU,285.27
Charing Cross SU,585.94
Chelsea & Wminster SU,327.21
Croydon SU,465.89
Epsom SU,351.83


## Add unit length of stay to summary dataframe

In [11]:
summary = summary.merge(hospital_info, left_on='Preferred_unit_name', right_on='Unit', how='left')

In [12]:
summary.drop(['Use', 'Capacity'], inplace=True, axis=1)

## Add calculated occupancy

Assume 45.2% use ASU (ASU admissions in 2019/19 SNAP / all LSOA admissions, with correction for use of Eposm which is not in London ASU)

In [13]:
require_asu = 0.452
summary['calc_occupancy'] = \
    summary['admissions'] * summary['los_mean'] / 365 * require_asu

## Set index, round output, and view final output

In [14]:
summary.set_index('Unit', inplace=True)
summary = summary.round(1);

In [15]:
summary

,admissions,Postcode,los_mean,calc_occupancy
Unit,,,,
Barnet General SU,285.3,EN53DJ,27.3,9.6
Charing Cross SU,585.9,W68RF,40.0,29.0
Chelsea & Wminster SU,327.2,SW109NH,29.5,12.0
Croydon SU,465.9,CR77YE,39.6,22.8
Epsom SU,351.8,KT187EG,21.6,9.4
Hillingdon SU,494.7,UB83NN,33.7,20.6
Homerton SU,297.3,E96SR,54.7,20.1
Kingston SU,341.1,KT27QB,30.4,12.8
King’s College SU,318.4,SE59RS,35.1,13.8


## Save

In [16]:
summary.to_csv('summary_unit_statistics.csv')